In [1]:
import os
import cv2
import random
# import pathlib
import pathlib
import numpy as np
import multiprocessing
from sklearn import svm


from matplotlib import pyplot as plt
from skimage.morphology import skeletonize
from skimage.feature import hog
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [2]:
lp_file_path = "./files/LP_COORDS.npz"

In [3]:
cpu_count = multiprocessing.cpu_count()

In [4]:
def get_lp_coord(image_name):
    image_name = os.path.basename(str(image_name))
    lp_coords = np.load(lp_file_path, allow_pickle = True)['coords'].tolist()
    (lp_x1, lp_y1, lp_x2, lp_y2) = lp_coords[image_name]
    lp_w, lp_h = lp_x2 - lp_x1, lp_y2 - lp_y1
    lp_xc = int((lp_x1 + lp_x2)/2)
    return (lp_x1, lp_y1, lp_x2, lp_y2), (lp_w, lp_h, lp_xc)

In [5]:
def get_all_images(input_dir, ext = "jpg", shuffle = True):
    image_paths = [i for i in pathlib.Path(input_dir).rglob("*.jpg")] #(".".join(["*", ext]))]
    if shuffle:
        random.shuffle(image_paths)
    return image_paths

In [6]:
def mean_smooth(input_array, win_size = 10):
    step = win_size //2 
    len_array = len(input_array)
    return_array = []
    for index, value in enumerate(input_array):
        i_range = [ max(0, index - step), min(len_array, index + step)]
        return_array.append(np.mean(input_array[i_range[0] : i_range[1] + 1]))
    return np.asarray(return_array)

In [40]:
def list_split(input_list, ratio = [0.5, 0.5], shuffle = False):
    assert sum(ratio) == 1, print("sum of ratio should be 1")
    input_list = sorted(input_list)
    len_list = len(input_list)
    
    split_len = []
    for i in ratio[:-1]:
        split_len.append(round(len_list*i))
    
    return_list = []
    temp = 0
    for i in split_len:
        start, end = temp, temp + i
        temp += i
        return_list.append(input_list[start: end])
    return_list.append(input_list[temp:])
        
    return return_list
    
    

In [6]:
def imread(file_name, is_gray = False, resize = False, width = None, height = None):
    if is_gray:
        image = cv2.imread(str(file_name), flags = cv2.IMREAD_GRAYSCALE)
    else:
        image = cv2.cvtColor(cv2.imread(str(file_name)), cv2.COLOR_BGR2RGB)
    
    if resize:
        h, w = image.shape[:2]
        if width == None and height != None:
            width = int(height/h*w)
        elif width != None and height == None:
            height = int(width/w*width)

        image = cv2.resize(image, (width, height))
        
    return image


def resize(input_image, is_gray = True, width = None, height = None):
    if is_gray:
        image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    h, w = image.shape[:2]
    if width == None and height != None:
        width = int(height/h*w)
    elif width != None and height == None:
        height = int(width/w*width)

    image = cv2.resize(image, (width, height))
    
    return image


In [7]:
winSize = (256, 128) 
blockSize = (64, 64)
blockStride = (32, 32)
cellSize = (32,32)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 1
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True

cvhog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,
                        derivAperture,winSigma,histogramNormType,L2HysThreshold,
                        gammaCorrection,nlevels,signedGradients)

In [8]:
def roi_extraction_1(input_image):
#     image_name = input_image.name
    image = imread(str(input_image))
    h, w = image.shape[:2]
    (x1, y1, x2, y2), (lp_w, lp_h, _) = get_lp_coord(image_name)
    
    roi_x1, roi_x2 = max(0, int(x1 - 0.8*lp_w)), min(int(x2 + 0.8*lp_w), w)
    roi_y1, roi_y2 = max(0, int(y1 - 0.32*(roi_x2 - roi_x1))), y1
    
    return [x1, y1, x2, y2], [roi_x1, roi_y1, roi_x2, roi_y2], image[roi_y1:roi_y2, roi_x1:roi_x2]
    

In [ ]:
def roi_extraction_2(input_image):
#     image_name = input_image.name
    image = imread(str(input_image))
    h, w = image.shape[:2]
    (x1, y1, x2, y2), (lp_w, lp_h, _) = get_lp_coord(image_name)
    
    roi_x1, roi_x2 = max(0, int(x1 - 0.84*lp_w)), min(int(x2 + 0.84*lp_w), w)
    roi_y1, roi_y2 = max(0, int(y1 - 3.67*lp_h)), min(int(y2 + 1.53*lp_h), h)
    
    return [x1, y1, x2, y2], [roi_x1, roi_y1, roi_x2, roi_y2], image[roi_y1:roi_y2, roi_x1:roi_x2]
    

In [ ]:
def roi_extraction_with_hood(input_image):
    
#     image_name = input_image.name
    image = imread(str(input_image))
    image_copy = image.copy()
    
    im_h, im_w = image.shape[:2]
    (lp_x1, lp_y1, lp_x2, lp_y2), (lp_w, lp_h, lp_xc) = get_lp_coord(input_image)

    roi_y2 = lp_y1
    
    sr_x1, sr_x2 = int(max(0, lp_x1 - 1.5*lp_w)), int(min(im_w, lp_x2 + 1.5*lp_w))
    sr_y1, sr_y2 = int(max(0, lp_y1 - 1.45*lp_w)), int(min(im_h, lp_y1 - 0.8*lp_w))
    sr_w,  sr_h = sr_x2 - sr_x1, sr_y2 - sr_y1
    sr = cv2.cvtColor(image[sr_y1:sr_y2, sr_x1:sr_x2], cv2.COLOR_RGB2GRAY)
    
    
    # blur the image 
    sr_blur = cv2.GaussianBlur(sr, ksize=(5, 5), sigmaX=0, sigmaY=0)
    # sobel y
    sobel_sr_y = cv2.convertScaleAbs(cv2.Sobel(sr_blur, cv2.CV_16S, 0, 1, ksize = 3))

    # horizontal projection and normalize
    # 去除边缘图像左右两侧1/4的梯度信息，这部分信息可能包含车辆以外的梯度噪声，
    # 对挡风玻璃下沿的投影效果产生影响
    sobel_sr_y[:, :sobel_sr_y.shape[1]//4] = 0
    sobel_sr_y[:, sobel_sr_y.shape[1]//4*3:] = 0

    y_project = np.sum(sobel_sr_y, 1)
    y_project = y_project / np.max(y_project)
    y_project = mean_smooth(y_project, win_size = 10)

    # the roi_y1
    roi_y1 = np.argmax(y_project) + sr_y1
    
    raw_roi = cv2.cvtColor(image[roi_y1 : lp_y1, sr_x1 : sr_x2], cv2.COLOR_BGR2GRAY)
#     raw_roi_color = cv2.cvtColor(image[roi_y1 : lp_y1, sr_x1 : sr_x2], cv2.COLOR_BGR2RGB)

    # 如果车牌左边界向左移动1.25个车牌宽度后距离图片边界大于20 则左侧的右边界=搜索区域左边界+0.55个车牌宽度
    left_bound = sr_x1 + int(0.55*lp_w) if (lp_x1 - 1.25*lp_w) > 20 else 10
    left_region = cv2.cvtColor(image[roi_y1:lp_y1, sr_x1 : left_bound], cv2.COLOR_BGR2GRAY)

    # 如果车牌右边界向右移动1.25个车牌宽度后距离图片右边界大于20，则设定右侧边界的左边界=搜索区域右边界-0.55个车牌宽度
    right_bound = int(sr_x2 - 0.55*lp_w) if (image.shape[1] - (lp_x2 + 1.25*lp_w)) > 20 else int(raw_roi.shape[1] - 10)
    right_region = cv2.cvtColor(image[roi_y1:lp_y1, right_bound: sr_x2], cv2.COLOR_BGR2GRAY)

    
    
    left_region[:left_region.shape[0]//5,:] = 0
    left_region = cv2.GaussianBlur(left_region, ksize=(15, 15), sigmaX=0, sigmaY=0)

    kernel=np.array([[-1,0,1],
                      [-1,0,1],
                      [-1,0,1],])

    left_region = np.float32(left_region)
    left_region = cv2.filter2D(left_region, -1, kernel)
    left_region = cv2.convertScaleAbs(left_region) #
    left_region = cv2.threshold(left_region,11,1,cv2.THRESH_OTSU)[1]
    left_region = np.uint8(skeletonize(left_region)*255)
    left_region = cv2.blur(left_region, (1, 5)) #left_region = cv2.GaussianBlur(left_region, ksize=(15, 15), sigmaX=0, sigmaY=0)


    left_project = np.sum(left_region, 0)
    left_project = left_project / np.max(left_project)
    left_project = mean_smooth(left_project, win_size = 5)

    x_left = np.argmax(left_project) #- 10
    roi_x1 = x_left + sr_x1
    
    
    
    
    
    
    
    right_region[:right_region.shape[0]//5,:] = 0
    right_region = cv2.GaussianBlur(right_region, ksize=(15, 15), sigmaX=0, sigmaY=0)

    right_region = np.float32(right_region)
    right_region = cv2.filter2D(right_region, -1, kernel)
    right_region = cv2.convertScaleAbs(right_region) 
    right_region = cv2.threshold(right_region,11,1,cv2.THRESH_OTSU)[1]
    right_region = np.uint8(skeletonize(right_region)*255)
    right_region = cv2.blur(right_region, (1, 5))#cv2.GaussianBlur(right_region, ksize=(15, 3), sigmaX=0, sigmaY=0)

    right_project = np.sum(right_region, 0)
    right_project = right_project /np.max(right_project)
    right_project = mean_smooth(right_project, win_size = 5)

    x_right = np.argmax(right_project) #+ 10
    roi_x2 = x_right + right_bound
    
    
    
    dist = min(roi_x2 - lp_xc, lp_xc - roi_x1)
    roi_x1, roi_x2 = lp_xc - dist, lp_xc + dist
    
    return [], [], image_copy[roi_y1:roi_y2, roi_x1:roi_x2]

In [ ]:
def overall_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def mean_accuracy(y_true, y_pred):
    if not isinstance(y_true, np.ndarray):
        y_true = np.asarray(y_true)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.asarray(y_pred)

    unique_label = np.unique(y_true)
    per_class_acc = []
    for label in unique_label:
        index = np.where(y_true == label)
        per_class_acc.append(np.mean(y_true[index] == y_pred[index]))
    return np.mean(per_class_acc, dtype=np.float32)


def mean_prec(y_true, y_pred):
    return precision_score(y_true, y_pred, average = "macro")

def overall_prec(y_true, y_pred):
    if not isinstance(y_true, np.ndarray):
        y_true = np.asarray(y_true)
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.asarray(y_pred)

    return np.mean(y_true == y_pred, dtype=np.float32)


def mean_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average = "macro")